In [55]:
import urllib
import json
import shapefile
from shapely.geometry import shape, Point
from shapely.ops import transform
import pyproj
import re

In [56]:
# FETCH COUNT SITES FROM HIGHWAYS ENGLAND API
url_text = "https://webtris.highwaysengland.co.uk/api/v1/sites"
with urllib.request.urlopen(url_text) as url:
    data = json.loads(url.read().decode())
sites = data['sites']

# HELPER FUNCTIONS TO UNDERSTAND DATA
def roadForSite(site):
    expression_desc = r"[A-Z]\d{1,3}[A-Z]*";
    expression_name = r"((?<=at |on )|(?<=link ))[A-Z]\d{1,3}[A-Z]*"

    match = re.search(expression_desc, site["Description"])
    if match: return match.group()

    match = re.search(expression_name, site["Name"])
    if match: return match.group()

    return "Unknown Road"


def isARoad(site):
    road = roadForSite(site)
    return road.startswith("A")

def isMotorway(site):
    road = roadForSite(site)
    return road.startswith("M")

def isActive(site):
    return site["Status"] == "Active"

def motorwaySites(sites):
    iterator=filter(isMotorway, sites);
    return list(iterator);

def aroadSites(sites):
    iterator=filter(isARoad, sites)
    return list(iterator)

def activeSites(sites):
    iterator=filter(isActive, sites)
    return list(iterator)

In [57]:
# COLLECT STATISTICS
motorway_sites = motorwaySites(sites)
aroad_sites = aroadSites(sites)
all_active_sites = activeSites(sites)
active_msites = activeSites(motorway_sites)
active_asites = activeSites(aroad_sites)

print(f'Sites (active): \t\t\t{len(sites)}({len(all_active_sites)})')
print(f'Motorway Sites (active): \t{len(motorway_sites)}({len(active_msites)})')
print(f'A-Road Sites (active): \t\t{len(aroad_sites)}({len(active_asites)})')


Sites (active): 			19364(11212)
Motorway Sites (active): 	12818(7774)
A-Road Sites (active): 		4184(3368)


In [72]:
# READ THE SHAPEFILE
#shapefile_location = './GLTLA_DEC_2022_EW_BFC_7755317155209021260/GLTLA_DEC_2022_EW_BFC.shp'
shapefile_location = "./UK_LocalDistrictAuthorities2023/LAD_MAY_2023_UK_BFC.shp"
shp = shapefile.Reader(shapefile_location)
all_shapes = shp.shapes() # get all the polygons
all_records = shp.records()

# DEFINE PROJECTION TO CONVERT SHAPEFILE COORDINATES TO LONG/LAT VALUES
project = pyproj.Transformer.from_proj(
    pyproj.Proj('epsg:27700'), # source coordinate system
    pyproj.Proj('epsg:4326')) # destination coordinate system

def assignSiteToLA(site, dictionary, startIndex = 0):
    for i in range(0,len(all_shapes)):
        index = (i+startIndex)%len(all_shapes)
        boundary = transform(project.transform, shape(all_shapes[index]))
        if boundary.contains(Point(site["Latitude"], site["Longitude"])):
            la_code =  all_records[index][0]
            dictionary[la_code].append(site)
            return index

In [75]:
# DATA STRUCTURE TO STORE COUNT SITES AND THEIR LA'S
la_dictionary = {record[0]: [] for record in all_records}

active_asites.sort(key=lambda site: site["Description"])

# ASSIGN COUNT SITES TO LOCAL AUTHORITY (TAKES ~4 HOURS TO RUN)
previous_shape_index = 0;
for index, site in enumerate(active_asites):
    previous_shape_index = assignSiteToLA(site, la_dictionary, previous_shape_index)
    print(f"DONE {index + 1} sites of {len(active_asites)}.\t{int(float(100)*index/len(active_asites))}%.")

# SAVE DATA STRUCTURE TO JSON FILE TO AVOID RERUNNING
with open('la_countsite_dict.json', 'w') as dictionary_json:
    json.dump(la_dictionary, dictionary_json)

DONE 1 sites of 3368.	0%.
DONE 2 sites of 3368.	0%.
DONE 3 sites of 3368.	0%.
DONE 4 sites of 3368.	0%.
DONE 5 sites of 3368.	0%.
DONE 6 sites of 3368.	0%.
DONE 7 sites of 3368.	0%.
DONE 8 sites of 3368.	0%.
DONE 9 sites of 3368.	0%.
DONE 10 sites of 3368.	0%.
DONE 11 sites of 3368.	0%.
DONE 12 sites of 3368.	0%.
DONE 13 sites of 3368.	0%.
DONE 14 sites of 3368.	0%.
DONE 15 sites of 3368.	0%.
DONE 16 sites of 3368.	0%.
DONE 17 sites of 3368.	0%.
DONE 18 sites of 3368.	0%.
DONE 19 sites of 3368.	0%.
DONE 20 sites of 3368.	0%.
DONE 21 sites of 3368.	0%.
DONE 22 sites of 3368.	0%.
DONE 23 sites of 3368.	0%.
DONE 24 sites of 3368.	0%.
DONE 25 sites of 3368.	0%.
DONE 26 sites of 3368.	0%.
DONE 27 sites of 3368.	0%.
DONE 28 sites of 3368.	0%.
DONE 29 sites of 3368.	0%.
DONE 30 sites of 3368.	0%.
DONE 31 sites of 3368.	0%.
DONE 32 sites of 3368.	0%.
DONE 33 sites of 3368.	0%.
DONE 34 sites of 3368.	0%.
DONE 35 sites of 3368.	1%.
DONE 36 sites of 3368.	1%.
DONE 37 sites of 3368.	1%.
DONE 38 si